# Convert MNI cooridnates to VBM t-values

In [1]:
import os, sys
import numpy as np
import pandas as pd
import scipy.spatial
import math
import feather
from scipy.stats import t, spearmanr, norm, wilcoxon, mannwhitneyu, ttest_ind
import matplotlib
from matplotlib.pyplot import scatter
from statsmodels.sandbox.stats.multicomp import multipletests
from mpl_toolkits.mplot3d import Axes3D
import nibabel as nib

In [2]:
#converts mni coordinate to voxel cooridnate
def mni152_to_voxel(coord, mat):
    tr = np.matrix(mat).getI()
    mni = np.ones(4)
    for i in range(3):
        mni[i] = coord[i]
    vox = np.dot(tr, mni)
    return vox.round().tolist()[0][0:3]


In [3]:
#returns value at a given coordinate
def coord2lab_and_dist(coord, affine, ldat):
    vox = mni152_to_voxel(coord, affine)
    if not pd.isna(vox[0]):
        val = ldat[int(vox[0]), int(vox[1]), int(vox[2])]
        return round(val,3)
    #return float('NaN'), float('NaN')
    #return None, None
    return None

In [18]:
#options: nifti file, options:
# 0004: C9ORF
# 0008: GRN
# 0012: MAPT
gene2num = {"C9ORF":"0004", "GRN":"0008", "MAPT":"0012"}

ftd_gene = "MAPT"

nifti_file = "../img_data/spmT_"+gene2num[ftd_gene]+"_mni.nii.gz"
selected_samples_file="../data/selected_samples_lh_for_FTD_AIBS.csv"

In [19]:
##load a few nifit files:
#load nift file and get the data
#nifty file containing closest DK_label
img_data  = os.path.join('./' , nifti_file)
atrophy_map = nib.load(img_data)
atrophy_dat = atrophy_map.get_data()


In [20]:
sample_info = pd.read_csv(selected_samples_file, index_col=0)
sample_mni = sample_info.loc["corrected_mni_x":"corrected_mni_z"]
sample_info.head()

,well_id,structure_id,slab_num,slab_type,structure_acronym,structure_name,polygon_id,mri_voxel_x,mri_voxel_y,mri_voxel_z,mni_x,mni_y,mni_z,corrected_mni_x,corrected_mni_y,corrected_mni_z,sampleID,probe2label,probe2dist
0,10964,4023,2,CX,SFG-m,"superior frontal gyrus, left, medial bank of g...",1266467,96,55,47,-5.0,44.0,54.0,-4.456913,42.626309,53.306473,H0351_1009,192,0.000000
1,10965,4039,2,CX,trIFG,"inferior frontal gyrus, triangular part, left",1266555,128,113,44,-37.0,47.0,-4.0,-37.014483,42.918867,-0.301717,H0351_1009,206,1.000000
2,10966,4051,3,CX,MOrG,"medial orbital gyrus, left",1265851,100,137,59,-9.0,32.0,-28.0,-9.535526,30.015430,-24.276204,H0351_1009,148,0.000000
3,10972,4224,2,CX,CgGf-i,"cingulate gyrus, frontal part, left, inferior ...",1266689,96,110,46,-5.0,45.0,-1.0,-5.445986,42.277137,1.581663,H0351_1009,102,0.000000
4,10973,4024,3,CX,SFG-l,"superior frontal gyrus, left, lateral bank of ...",1265853,114,63,62,-23.0,29.0,46.0,-18.855630,25.569178,43.817038,H0351_1009,192,1.414214


### Map to t-value!

In [21]:
my_affine=atrophy_map.affine
img_vals = atrophy_map.get_data()


In [22]:
#original
coords=sample_info.loc[:,"corrected_mni_x":"corrected_mni_z"]
original = pd.DataFrame(coords.apply(lambda x: coord2lab_and_dist(x, my_affine, img_vals), axis=1))
original.set_index(sample_info.well_id, inplace=True)
original.columns = ['original']

In [23]:
original.to_csv("../data/image_values_"+ftd_gene+".csv")